
# Notebook ReAct aplicada

Clase aplicada de ReAct con herramientas simples, trazabilidad y feedback loop de calidad.


In [1]:
from __future__ import annotations

import json
import os
import sys
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI

# Add project root to path
project_root = Path.cwd()
while not (project_root / "pyproject.toml").exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

# Add 02-prompting to path
prompting_path = project_root / "02-prompting"
if str(prompting_path) not in sys.path:
    sys.path.insert(0, str(prompting_path))

from common.rubrica import evaluar_salida

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("Falta OPENAI_API_KEY en .env")

model = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
client = OpenAI(api_key=api_key)

In [2]:

perfil = {
    "tipo_persona": "consultora de marca personal",
    "gustos": ["arte contemporaneo", "vino", "estrategia de negocio"],
    "estilo": "calida, analitica y directa",
    "contexto": "conversacion retomada despues de varios dias",
}

def analizar_perfil(p):
    return {
        "insights": [
            f"Intereses clave: {', '.join(p.get('gustos', []))}",
            "Conviene una apertura breve con pregunta concreta.",
        ],
        "tono": p.get("estilo", "calido"),
    }

def auditar_respeto(salida):
    texto = f"{salida.get('opener', '')} {salida.get('follow_up', '')}".lower()
    flags = [x for x in ["presion", "insistir", "explicito"] if x in texto]
    return {"ok": len(flags) == 0, "flags": flags}

analysis = analizar_perfil(perfil)
analysis


{'insights': ['Intereses clave: arte contemporaneo, vino, estrategia de negocio',
  'Conviene una apertura breve con pregunta concreta.'],
 'tono': 'calida, analitica y directa'}

In [3]:

prompt_react = f"""
Usa formato ReAct con esta secuencia:
Thought -> Action -> Observation -> Thought -> Action -> Observation -> Final Answer

Herramientas disponibles:
- analizar_perfil: {json.dumps(analysis, ensure_ascii=False)}
- auditar_respeto: se ejecuta despues de proponer mensaje

Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Genera JSON:
{{
  "trace": [
    {{"thought": "...", "action": "...", "observation": "..."}},
    {{"thought": "...", "action": "...", "observation": "..."}}
  ],
  "result": {{
    "opener": "...",
    "follow_up": "...",
    "why_it_works": ["...", "..."]
  }}
}}

No uses emojis.
"""

resp = client.chat.completions.create(
    model=model,
    temperature=0.5,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "Eres un agente ReAct con enfoque profesional y respetuoso."},
        {"role": "user", "content": prompt_react},
    ],
)

react_out = json.loads(resp.choices[0].message.content)
audit = auditar_respeto(react_out["result"])
eval_react = evaluar_salida(perfil, react_out["result"])

print(json.dumps(react_out, ensure_ascii=False, indent=2))
print(json.dumps(audit, ensure_ascii=False, indent=2))
print(json.dumps(eval_react, ensure_ascii=False, indent=2))


{
  "trace": [
    {
      "thought": "Es importante retomar la conversación de manera cálida y directa, considerando el tiempo que ha pasado.",
      "action": "Formular una pregunta concreta que invite a la reflexión y al diálogo sobre sus intereses.",
      "observation": "Esto puede ayudar a reanudar la conexión y mostrar interés genuino en su perspectiva."
    },
    {
      "thought": "Debo asegurarme de que el mensaje sea relevante para su perfil y sus gustos.",
      "action": "Proponer un tema que combine arte contemporáneo y estrategia de negocio, dado su interés en ambos.",
      "observation": "Esto puede generar un intercambio significativo y mantener la conversación centrada en sus pasiones."
    }
  ],
  "result": {
    "opener": "¿Cómo ha influido el arte contemporáneo en tu enfoque de estrategia de negocio últimamente?",
    "follow_up": "Me encantaría escuchar tus pensamientos sobre cómo estas dos áreas pueden interrelacionarse.",
    "why_it_works": [
      "Conecta 

In [4]:

prompt_refine = f"""
Perfil:
{json.dumps(perfil, ensure_ascii=False, indent=2)}

Salida actual:
{json.dumps(react_out, ensure_ascii=False, indent=2)}

Auditoria:
{json.dumps(audit, ensure_ascii=False, indent=2)}

Rubrica:
{json.dumps(eval_react, ensure_ascii=False, indent=2)}

Mejora result.opener y result.follow_up con el mismo estilo ReAct.
No uses emojis.
Devuelve JSON con trace y result.
"""

resp2 = client.chat.completions.create(
    model=model,
    temperature=0.3,
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": "Eres auditor y optimizador de agentes ReAct."},
        {"role": "user", "content": prompt_refine},
    ],
)

mejorada = json.loads(resp2.choices[0].message.content)
print(json.dumps(mejorada, ensure_ascii=False, indent=2))
print(json.dumps(evaluar_salida(perfil, mejorada["result"]), ensure_ascii=False, indent=2))


{
  "trace": [
    {
      "thought": "Es fundamental retomar la conversación de manera cálida y directa, considerando el tiempo que ha pasado.",
      "action": "Formular una pregunta que no solo invite a la reflexión, sino que también haga referencia a sus intereses específicos.",
      "observation": "Esto puede ayudar a reanudar la conexión y demostrar un interés genuino en su perspectiva."
    },
    {
      "thought": "Debo asegurarme de que el mensaje sea relevante para su perfil y sus gustos, especialmente en relación al arte contemporáneo y la estrategia de negocio.",
      "action": "Proponer un tema que combine sus pasiones de manera más explícita.",
      "observation": "Esto puede generar un intercambio significativo y mantener la conversación centrada en sus intereses."
    }
  ],
  "result": {
    "opener": "¿De qué manera crees que el arte contemporáneo puede inspirar nuevas estrategias en el mundo de los negocios?",
    "follow_up": "Me gustaría conocer tu opinión sobr